In [6]:
import boto3
import zipfile
import os
import sagemaker
import tensorflow as tf

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[03/27/25 05:41:58] INFO     Found credentials from IAM Role:                                   ]8;id=93182;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=653966;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


### Data Preprocessing

In [4]:
# Define dataset path
s3_bucket = "id-classifier-images"
s3_key = "images.zip"
local_zip_path = "/tmp/images.zip"
dataset_dir = "/tmp/dataset"

# Download and extract dataset from S3
s3 = boto3.client("s3")
# s3.download_file(s3_bucket, s3_key, local_zip_path)

# with zipfile.ZipFile(local_zip_path, "r") as zip_ref:
#     zip_ref.extractall(dataset_dir)

### Initialize SageMaker session and role

In [8]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()


### Define image parameters for model input

In [9]:
IMG_HEIGHT = 150  # Height of input images
IMG_WIDTH = 150   # Width of input images
BATCH_SIZE = 32   # Number of images per batch

### Create ImageDataGenerator for data augmentation and preprocessing

In [ ]:
data_gen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # Normalize pixel values